**Dependancies**

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os, sys, random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils
from keras.models import load_model
from keras.models import model_from_json

Using TensorFlow backend.


**Variables**

In [2]:
csv_data_dir = "../data/CSV"
np_data_dir = "../data/Numpy"
save_dir = "../logs"
dances = []
BATCH_SIZE = 1
N_TIMESTEPS = 20
N_EPOCHS = 15

** Pull Names of Dance Data **

In [3]:
def getFileNames():
    filenames = [f for f in os.listdir(csv_data_dir) if f.endswith('.csv')]
    for file in enumerate(filenames):
        filenames[file[0]] = file[1][:-7]
    return set(filenames)

** Pre-Process Data **

In [4]:
def pre_process_data(filename):
    #filename = os.path.join(data_dir)+filename
    
    pos_data = pd.read_csv(filename+"pos.csv")
    rot_data = pd.read_csv(filename+"rot.csv")

    #normalization force values from -1 to 1
    rot_data = rot_data/180.0

    #Add the root (hip) data for spacial movement
    rot_data['Hips.pos.x'] = pos_data.pop('Hips.x')
    rot_data['Hips.pos.y'] = pos_data.pop('Hips.y')
    rot_data['Hips.pos.z'] = pos_data.pop('Hips.z')

    #Making movement relative to an origin of 0,0,0 for consistancy within different dances
    rot_data['Hips.pos.x'] = rot_data['Hips.pos.x'] + (-1*rot_data['Hips.pos.x'][0])
    rot_data['Hips.pos.y'] = rot_data['Hips.pos.y'] + (-1*rot_data['Hips.pos.y'][0])
    rot_data['Hips.pos.z'] = rot_data['Hips.pos.z'] + (-1*rot_data['Hips.pos.z'][0])

    time = rot_data.pop('time') #maybe change to time change value instead? To indicate speed
    data = rot_data.copy()

    return data

** Load Data and Separate Into Samples **

In [5]:
def get_sample_data(filename):
    data = pre_process_data(os.path.join(csv_data_dir, filename))
    N_ROWS = data.values.shape[0]
    N_COLOMNS = data.values.shape[1]
    
    data = data.iloc[:].values
    dataX = []
    dataY = []

    for i in range(0, N_ROWS - N_TIMESTEPS, 1):
        seqIn = data[i: i+N_TIMESTEPS]
        seqOut = data[i+N_TIMESTEPS : i+N_TIMESTEPS+1]
        dataX.append(seqIn)
        dataY.append(seqOut)

    return dataX, dataY

** Set-Up Model **

In [6]:
def create_model(N_COLOMNS, stateful):
    model = keras.Sequential()
    model.add(keras.layers.LSTM(256, activation='relu', 
                                input_shape = (N_TIMESTEPS, N_COLOMNS), 
                                batch_size = BATCH_SIZE, 
                                return_sequences=True, 
                                stateful=stateful))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.LSTM(256, activation='relu', stateful=stateful))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(N_COLOMNS, activation='tanh'))
    return model

** Train Model **

In [2]:
def train_model():
    model = create_model(165, True)
    model.compile(optimizer='adam', loss='mse') #metrics=['accuracy']
    print(model.summary())
    
    #define the checkpoint
    filepath = os.path.join(save_dir, "com-weights-improvement-{epoch:02d}-{loss:.4f}.hdf5")
    checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose = 1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]
    
    loadedX = os.path.join(np_data_dir, "Combined-X-"+str(N_TIMESTEPS))
    loadedY = os.path.join(np_data_dir, "Combined-Y-"+str(N_TIMESTEPS))
    
    if not (os.path.exists(loadedX+".npy") and os.path.exists(loadedY+".npy")):
        print("create")
    
        dances = list(getFileNames())   
        X = []
        Y = []

        for dance in dances:
            x, y = get_sample_data(dance)
            print(dance)
            X.append(x)
            Y.append(Y)

        #X shape [samples, timesteps, features]
        #Y shape [samples, 1, features]
        X, Y = np.array(X), np.array(Y)
        print(X.shape)
        print(Y.shape)

        N_SAMPLES = len(X)
        Y = np.reshape(Y, (N_SAMPLES, N_COLOMNS))
        
        print("saving")
        np.save(loadedX, X)
        np.save(loadedY, Y)
    
    
    X = np.load(loadedX+".npy")
    Y = np.load(loadedY+".npy")
    print(X.shape)
    print(Y.shape)

    #train/fit the model

    model.fit(X, Y, epochs = N_EPOCHS, callbacks=callbacks_list)
    model.save("model-Com-"+str(N_TIMESTEPS)+".h5") #arch+weight+optimizer state
    #json_string = model.to_jason() #architecture
    #model.save_weights("modelWeights-Com-"+str(N_TIMESTEPS)+".h5") #weights
    
    #model = load_model("model-Com-"+str(N_TIMESTEPS)+".h5")
    #model = model_from_json(json_string)
    #model.load_weights("modelWeights-Com-"+str(N_TIMESTEPS)+".h5")
train_model()

NameError: name 'create_model' is not defined

** Sample Model **

In [ ]:
def trim_weights():
    minFile = ""
    minLoss = 100
    '''test = {"weights-improvement-19-1.2765.hdf5",
            "weights-improvement-20-1.8434.hdf5",
            "weights-improvement-8-1.1234.hdf5"}'''
    for file in os.listdir("./"):
    #for file in test:
        if file.endswith(".hdf5"):
            string = file.split('-')
            value = (float)(os.path.splitext(string[len(string)-1])[0])
            if(minLoss>value):
                minLoss=value
                minFile = file
    return minFile